<a href="https://colab.research.google.com/github/galb4tosha/my-ml/blob/master/My_Diplom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка


In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!git clone https://github.com/tensorflow/models.git
!cd models/research; protoc object_detection/protos/*.proto --python_out=.
!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

In [0]:
!pip install -U --pre tensorflow=="2.*"
!pip install pycocotools

Requirement already up-to-date: tensorflow==2.* in /usr/local/lib/python3.6/dist-packages (2.2.0rc2)


In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1017522 sha256=b39dbf93aeb394a0d2e32cb19d10f2f060fd35a69c24a3fe241d71dc653605ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-mljzn0j7/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


bash: line 3: cd: models/research: No such file or directory


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import datetime
import cv2
import logging

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [0]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('models/research/object_detection/test_images/image1.jpg'),
 PosixPath('models/research/object_detection/test_images/image2.jpg')]

In [0]:
model_name = 'ssd_mobilenet_v1_coco_2018_01_28'
detection_model = load_model(model_name)

76546048/76541073 [==============================] - 1s 0us/step
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Some methods


In [0]:
def load_image(image_file_name):
    image = Image.open(image_file_name)
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [0]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [0]:
def logger(frame,out_file):
  f = open(out_file,'a')
  f.write(f"Warning, in {frame} was a person.\n")
  f.close()

My methods


In [0]:
def find_persons(img,model):
  start_time = datetime.datetime.now()
  image_np = np.array(img)
  output_dict = run_inference_for_single_image(model, image_np)
  print(datetime.datetime.now() - start_time)
  is_person = False
  for i in range(len(output_dict['detection_classes'])):
    if output_dict['detection_classes'][i] == 1 and output_dict['detection_scores'][i] > 0.3:
      is_person = True
  if is_person:
    vis_util.visualize_boxes_and_labels_on_image_array(
        img,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=(12, 8))
    plt.grid(False)
    plt.imshow(img)
  return is_person

In [0]:
def chek_persons_by_url(url):
  p = requests.get(url)
  out = open("...\img.jpg", "wb")
  out.write(p.content)
  out.close()
  img = load_image("...\img.jpg")
  return find_persons(img)

In [0]:
def chek_person_by_path(path):
  img = load_image(path)
  return find_persons(img)

In [0]:
def check_video(video, path_output_dir,model):
    # extract frames from a video and save to directory as 'x.png' where 
    # x is the frame index
    time_process = 1
    vidcap = cv2.VideoCapture(video)
    FPS = vidcap.get(cv2.CAP_PROP_FPS)
    frames_skip = int(FPS*time_process)
    count = 0
    while vidcap.isOpened():
      success, image = vidcap.read()
      count += 1
      temp = count % frames_skip
      if success:
        if temp == 0:
          if check_and_save(image,count,path_output_dir,model):
            logger(count,'/gdrive/My Drive/Colab Notebooks/Diplom/log.txt')
      else:
        break
      if count > 4000:
        break
    cv2.destroyAllWindows()
    vidcap.release()

In [0]:
def check_and_save(frame, count_frame, path_output_dir,model):
  if find_persons(frame,model):
    cv2.imwrite(os.path.join(path_output_dir, '%d.png') % count_frame, frame)
    return True
  else:
    return False

Testing


In [0]:
video_path = "/gdrive/My Drive/Colab Notebooks/Diplom/Cam.mp4"
save_path = "/gdrive/My Drive/Colab Notebooks/Diplom/Frames"

In [0]:
check_video(video_path,save_path,detection_model)

0:00:00.108213
0:00:00.104487


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


0:00:00.110759
0:00:00.111996
0:00:00.110035
0:00:00.111601
0:00:00.104193
0:00:00.106167
0:00:00.103214
0:00:00.106718
0:00:00.102959
0:00:00.112991
0:00:00.108061
0:00:00.110589
0:00:00.110351
0:00:00.110541
0:00:00.101375
0:00:00.128389
0:00:00.104582
0:00:00.100846
0:00:00.117960
0:00:00.101013
0:00:00.100867
0:00:00.107781
0:00:00.114023
0:00:00.114788
0:00:00.108878
0:00:00.104583
0:00:00.112442
0:00:00.123617
0:00:00.103464
0:00:00.106158
0:00:00.105909
0:00:00.121916
0:00:00.105178
0:00:00.102892
0:00:00.103721
0:00:00.108065
0:00:00.108621
0:00:00.110785
0:00:00.111160
0:00:00.103363
0:00:00.103616
0:00:00.103835
0:00:00.105657
0:00:00.105090
0:00:00.102076
0:00:00.106793
0:00:00.112369
0:00:00.108067
0:00:00.103238
0:00:00.103413
0:00:00.102898
0:00:00.106861
0:00:00.109718
0:00:00.113068
0:00:00.105729
0:00:00.133951
0:00:00.112212
0:00:00.106046
0:00:00.110172
0:00:00.100720
0:00:00.100619
0:00:00.111379
0:00:00.101984
0:00:00.133867
0:00:00.105355
0:00:00.120732
0:00:00.11